# Wikipedia Retriever 

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
retriever = WikipediaRetriever(top_k_results=3,lang='en')

In [3]:
query="One piece"
docs=retriever.invoke(query)

In [4]:
docs

[Document(metadata={'title': 'One Piece', 'summary': 'One Piece (stylized in all caps) is a Japanese manga series written and illustrated by Eiichiro Oda. It follows the adventures of Monkey D. Luffy and his crew, the Straw Hat Pirates, as he explores the Grand Line in search of the mythical treasure known as the "One Piece" to become the next King of the Pirates.\nThe manga has been serialized in Shueisha\'s shōnen manga magazine Weekly Shōnen Jump since July 1997, with its chapters compiled in 112 tankōbon volumes as of July 2025. It was licensed for an English language release in North America and the United Kingdom by Viz Media and in Australia by Madman Entertainment. Becoming a media franchise, it has been adapted into a festival film by Production I.G, and an anime series by Toei Animation, which premiered in October 1999. Additionally, Toei has developed 14 animated feature films and one original video animation (OVA). Several companies have developed various types of merchandi

In [5]:
for i,doc in enumerate(docs):
    print('Result',i+1,'\n')
    print(doc.page_content)
    



Result 1 

One Piece (stylized in all caps) is a Japanese manga series written and illustrated by Eiichiro Oda. It follows the adventures of Monkey D. Luffy and his crew, the Straw Hat Pirates, as he explores the Grand Line in search of the mythical treasure known as the "One Piece" to become the next King of the Pirates.
The manga has been serialized in Shueisha's shōnen manga magazine Weekly Shōnen Jump since July 1997, with its chapters compiled in 112 tankōbon volumes as of July 2025. It was licensed for an English language release in North America and the United Kingdom by Viz Media and in Australia by Madman Entertainment. Becoming a media franchise, it has been adapted into a festival film by Production I.G, and an anime series by Toei Animation, which premiered in October 1999. Additionally, Toei has developed 14 animated feature films and one original video animation (OVA). Several companies have developed various types of merchandising and media, such as a trading card game a

# Vector Store Retriever

In [6]:
%pwd

'c:\\Users\\ROSHAN\\OneDrive\\Desktop\\Vs Code\\Python\\lang\\Retrievers'

In [8]:
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv


load_dotenv()

embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [9]:
docs = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [10]:
vector_store=Chroma(embedding_function=embedding_model,
                    persist_directory='../VectorStore/chroma_db',
                    collection_name='retriever'
                    )

C:\Users\ROSHAN\AppData\Local\Temp\ipykernel_5404\3214272978.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_store=Chroma(embedding_function=embedding_model,


In [11]:
vector_store.add_documents(documents=docs)

['1896c9e4-4926-4ee3-8fd6-b57d62927cd2',
 'd5ec520e-88d9-42d3-ad28-3a774a4360b9',
 '154c381e-f7ce-4b0e-b0da-947dd03372f2',
 'bb88dd36-4573-4f9f-b603-9ada82244dcf']

In [12]:
# vector store retriever

retriever = vector_store.as_retriever(search_kwargs={'k':2})

query='what is  chroma used for'

In [13]:
res = retriever.invoke(query)
res

[Document(metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'),
 Document(metadata={}, page_content='LangChain helps developers build LLM applications easily.')]

# "mmr" (maximum marginal relevance)

In [14]:
from langchain_community.vectorstores import FAISS

In [15]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [17]:
vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model, 
)

vector_store

In [22]:
# mmr retriever with lambda_mult=1 ...would behave same as default similarity_search...

retriever  = vector_store.as_retriever(search_type='mmr',search_kwargs={'k':3,'lambda_mult':1})


In [23]:
query = 'what is langchain'

res = retriever.invoke(query)

res

[Document(id='13b888d4-8dca-45ac-9a12-173bd12a5ddc', metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.'),
 Document(id='a56fe57e-2582-4376-8394-ba9a720a0795', metadata={}, page_content='LangChain is used to build LLM based applications.'),
 Document(id='0cd082ae-9903-44aa-853d-57eb7d031ed8', metadata={}, page_content='LangChain makes it easy to work with LLMs.')]

In [24]:
# mmr retriever with lambda_mult=0.5

retriever  = vector_store.as_retriever(search_type='mmr',search_kwargs={'k':3,'lambda_mult':0.5})


In [26]:
query = 'what is langchain'

res = retriever.invoke(query)

res

[Document(id='13b888d4-8dca-45ac-9a12-173bd12a5ddc', metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.'),
 Document(id='a56fe57e-2582-4376-8394-ba9a720a0795', metadata={}, page_content='LangChain is used to build LLM based applications.'),
 Document(id='a3320fe1-7156-4c48-baef-7846f0d860f8', metadata={}, page_content='Embeddings are vector representations of text.')]

# Multiquery Retriever